In [2]:
import simpy
import numpy as np
import random

# server class
class Server:
    def __init__(self, env, dc_config):
        self.env = env
        self.dc_config = dc_config
        self.machine = simpy.Resource(env, capacity=1)
        self.latest_task_end_time = 0

    def can_accept_task(self, task):
        return self.dc_config['accumulated_ram'] >= task['required_ram'] and \
               self.dc_config['accumulated_storage'] >= task['required_storage']

    def process(self, task):
        if self.can_accept_task(task):
            with self.machine.request() as req:
                yield req
                yield self.env.timeout(task['duration'] / self.dc_config['processing_power'])
                self.latest_task_end_time = self.env.now
                self.dc_config['accumulated_ram'] -= task['required_ram']
                self.dc_config['accumulated_storage'] -= task['required_storage']
        else:
            raise Exception("Server cannot handle this task.")

# Parameters
edge_datacenters = [
    {
        'num_datacenter': 1,
        'host_ram': 10,
        'num_hosts': 1,
        'storage': 0.5,
        'bandwidth': 5,
        'processing_power': 250000,
        'vm_number': 30,
        'accumulated_ram': 0.5,
        'accumulated_storage': 0.5,
        'cost': 0.022,
    }
] * 2  # Two edge datacenters

cloud_datacenters = [
    {
        'num_datacenter': 1,
        'host_ram': 20,
        'num_hosts': 1,
        'storage': 1,
        'bandwidth': 10,
        'processing_power': 1000000,
        'vm_number': 30,
        'accumulated_ram': 2,
        'accumulated_storage': 10,
        'cost': 0.051,
    }
] * 2  # Two cloud datacenters

task_sizes = list(range(100, 1001, 100))
average_task_length = list(range(100, 1001, 100))
file_size = list(range(200, 401, 100))
output_size = [300]


class Flower:
    def __init__(self, decision_variables, lower_bound, upper_bound):
        self.position = np.random.uniform(
            low=lower_bound, high=upper_bound, size=decision_variables)
        self.fitness = np.inf


class SimulatedAnnealing:
    def __init__(self, fitness_function, initial_solution, initial_temperature, final_temperature, cooling_rate):
        self.fitness_function = fitness_function
        self.current_solution = initial_solution
        self.current_fitness = self.fitness_function(self.current_solution)
        self.best_solution = self.current_solution
        self.best_fitness = self.current_fitness
        self.temperature = initial_temperature
        self.final_temperature = final_temperature
        self.cooling_rate = cooling_rate

    def accept(self, candidate_fitness):
        if candidate_fitness < self.current_fitness:
            return True
        else:
            delta = self.current_fitness - candidate_fitness
            probability = np.exp(delta / self.temperature)
            return np.random.rand() < probability

    def cool_down(self):
        if self.temperature > self.final_temperature:
            self.temperature *= self.cooling_rate


class FlowerPollination:
    def __init__(self, fitness_function, population_size=100, decision_variables=10, lower_bound=0, upper_bound=1, generations=1000, switch_prob=0.8, gamma=0.1, beta=1.5):
        self.population = [Flower(
            decision_variables, lower_bound, upper_bound) for _ in range(population_size)]
        self.global_best = None
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.decision_variables = decision_variables
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.generations = generations
        self.switch_prob = switch_prob
        self.gamma = gamma
        self.beta = beta

    def levy_flight(self, beta):
        sigma = (np.math.gamma(1 + beta) * np.sin(np.pi * beta / 2) /
                 (np.math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u = np.random.normal(0, sigma, size=self.decision_variables)
        v = np.random.normal(0, 1, size=self.decision_variables)
        step = u / abs(v) ** (1 / beta)
        return step

    def global_pollination(self, flower):
        step_size = self.gamma * self.levy_flight(self.beta)
        new_position = flower.position + step_size * \
            (flower.position - self.global_best.position)
        return new_position

    def local_pollination(self, flower):
        flower_j = np.random.choice(self.population)
        epsilon = np.random.uniform(
            low=0, high=1, size=self.decision_variables)
        new_position = flower.position + epsilon * \
            (flower_j.position - flower.position)
        return new_position

    def pollination(self, flower):
        r = np.random.uniform(low=0, high=1)
        if r < self.switch_prob:
            new_position = self.global_pollination(flower)
        else:
            new_position = self.local_pollination(flower)
        return np.clip(new_position, self.lower_bound, self.upper_bound)

    def find_global_best(self):
        for flower in self.population:
            if flower.fitness < self.global_best.fitness:
                self.global_best = flower

    def optimize(self):
        self.global_best = self.population[0]
        self.global_best.fitness = self.fitness_function(self.global_best.position)
        sa = SimulatedAnnealing(self.fitness_function, self.global_best.position, initial_temperature=10, final_temperature=0.001, cooling_rate=0.9)

        for g in range(self.generations):
            for flower in self.population:
                flower.fitness = self.fitness_function(flower.position)
                self.find_global_best()

                new_position = self.pollination(flower)
                new_fitness = self.fitness_function(new_position)

                if sa.accept(new_fitness):
                    flower.position = new_position
                    flower.fitness = new_fitness

            sa.cool_down()  # cool down the temperature after each generation
            self.find_global_best()
            #print(f"Generation: {g}, Best fitness: {self.global_best.fitness}")

        return self.global_best

# Fitness Function
class FitnessFunction:
    def __init__(self, tasks, servers):
        self.tasks = tasks
        self.servers = servers
        self.makespans = {}

    def __call__(self, solution):
        task_times = []
        for i, task in enumerate(self.tasks):
            server = self.servers[solution[i]]
            processing_time = task['duration'] / server.dc_config['processing_power']
            task_times.append(server.latest_task_end_time + processing_time)
            server.latest_task_end_time += processing_time

        max_time = max(task_times)
        self.makespans.setdefault(len(self.tasks), []).append(max_time)
        return max_time

    def get_average_cost(self):
        return np.mean([server.dc_config['cost'] for server in self.servers])

    def get_average_utilization(self):
        return np.mean([server.latest_task_end_time for server in self.servers])

# Initialize Environment and Servers
env = simpy.Environment()
edge_servers = [Server(env, dc) for dc in edge_datacenters]
cloud_servers = [Server(env, dc) for dc in cloud_datacenters]

# Optimization
for task_size in task_sizes:
    num_tasks_per_size = task_size
    tasks = [task_size] * num_tasks_per_size
    print(f"Optimizing for cloud servers with task size: {task_size}, num_tasks: {num_tasks_per_size}")
    fitness_function = FitnessFunction(tasks, cloud_servers)
    fp = FlowerPollination(fitness_function, decision_variables=len(tasks), lower_bound=0, upper_bound=len(cloud_servers)-1, generations=100)
    best_flower = fp.optimize()
    makespan = best_flower.fitness
    average_cost = fitness_function.get_average_cost()
    average_utilization = fitness_function.get_average_utilization()
    print(f"Average makespan for cloud servers with task size {task_size}: {np.mean(fitness_function.makespans[task_size])}")
    print(f"Average resource utilization for cloud servers with task size {task_size}: {average_utilization}")
    print(f"Average cost for cloud servers with task size {task_size}: {average_cost}")

    print(f"Optimizing for edge servers with task size: {task_size}, num_tasks: {num_tasks_per_size}")
    fitness_function = FitnessFunction(tasks, edge_servers)
    fp = FlowerPollination(fitness_function, decision_variables=len(tasks), lower_bound=0, upper_bound=len(edge_servers)-1, generations=100)
    best_flower = fp.optimize()
    makespan = best_flower.fitness
    average_cost = fitness_function.get_average_cost()
    average_utilization = fitness_function.get_average_utilization()
    print(f"Average makespan for edge servers with task size {task_size}: {np.mean(fitness_function.makespans[task_size])}")
    print(f"Average cost for edge servers with task size {task_size}: {average_cost}")
    print(f"Average resource utilization for edge servers with task size {task_size}: {average_utilization}")


Optimizing for cloud servers with task size: 100, num_tasks: 100


TypeError: list indices must be integers or slices, not numpy.float64

In [7]:
import simpy
import numpy as np

# Server Class
class Server:
    def __init__(self, env, dc_config):
        self.env = env
        self.dc_config = dc_config
        self.machine = simpy.Resource(env, capacity=1)
        self.latest_task_end_time = 0
        self.dc_config['accumulated_bandwidth'] = dc_config['bandwidth']
        self.dc_config['accumulated_file_size'] = dc_config['storage']
        self.dc_config['accumulated_output_size'] = dc_config['storage']

    def can_accept_task(self, task):
        return self.dc_config['accumulated_ram'] >= task['required_ram'] and \
               self.dc_config['accumulated_storage'] >= task['required_storage'] and \
               self.dc_config['accumulated_bandwidth'] >= task['required_bandwidth']

    def process(self, task):
        if self.can_accept_task(task):
            with self.machine.request() as req:
                yield req
                yield self.env.timeout(task['duration'] / self.dc_config['processing_power'])
                self.latest_task_end_time = self.env.now
                self.dc_config['accumulated_ram'] -= task['required_ram']
                self.dc_config['accumulated_storage'] -= task['required_storage']
                self.dc_config['accumulated_bandwidth'] -= task['required_bandwidth']
                self.dc_config['accumulated_file_size'] -= task['file_size']
                self.dc_config['accumulated_output_size'] -= task['output_size']
        else:
            raise Exception("Server cannot handle this task.")

# Function to generate tasks
def generate_tasks(num_tasks=1000):
    tasks = []
    for _ in range(num_tasks):
        task = {
            'duration': np.random.randint(100, 1000),
            'required_ram': np.random.uniform(0.1, 1),
            'required_storage': np.random.uniform(0.1, 1),
            'required_bandwidth': np.random.uniform(0.1, 1),
            'file_size': np.random.uniform(0.1, 1),
            'output_size': np.random.uniform(0.1, 1),
        }
        tasks.append(task)
    return tasks

# Configuration of Data Centers
edge_datacenters = [
    {
        'num_datacenter': 1,
        'host_ram': 10,
        'num_hosts': 1,
        'storage': 0.5,
        'bandwidth': 5,
        'processing_power': 250000,
        'vm_number': 30,
        'accumulated_ram': 0.5,
        'accumulated_storage': 0.5,
        'cost': 0.022,
    }
] * 2

cloud_datacenters = [
    {
        'num_datacenter': 1,
        'host_ram': 20,
        'num_hosts': 1,
        'storage': 1,
        'bandwidth': 10,
        'processing_power': 1000000,
        'vm_number': 30,
        'accumulated_ram': 2,
        'accumulated_storage': 10,
        'cost': 0.051,
    }
] * 2

# Initialize environment and servers
env = simpy.Environment()
edge_servers = [Server(env, dc) for dc in edge_datacenters]
cloud_servers = [Server(env, dc) for dc in cloud_datacenters]

class Flower:
    def __init__(self, decision_variables, lower_bound, upper_bound):
        self.position = np.random.uniform(
            low=lower_bound, high=upper_bound, size=decision_variables)
        self.fitness = np.inf


class SimulatedAnnealing:
    def __init__(self, fitness_function, initial_solution, initial_temperature, final_temperature, cooling_rate):
        self.fitness_function = fitness_function
        self.current_solution = initial_solution
        self.current_fitness = self.fitness_function(self.current_solution)
        self.best_solution = self.current_solution
        self.best_fitness = self.current_fitness
        self.temperature = initial_temperature
        self.final_temperature = final_temperature
        self.cooling_rate = cooling_rate

    def accept(self, candidate_fitness):
        if candidate_fitness < self.current_fitness:
            return True
        else:
            delta = self.current_fitness - candidate_fitness
            probability = np.exp(delta / self.temperature)
            return np.random.rand() < probability

    def cool_down(self):
        if self.temperature > self.final_temperature:
            self.temperature *= self.cooling_rate


class FlowerPollination:
    def __init__(self, fitness_function, population_size=100, decision_variables=10, lower_bound=0, upper_bound=1, generations=1000, switch_prob=0.8, gamma=0.1, beta=1.5):
        self.population = [Flower(
            decision_variables, lower_bound, upper_bound) for _ in range(population_size)]
        self.global_best = None
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.decision_variables = decision_variables
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.generations = generations
        self.switch_prob = switch_prob
        self.gamma = gamma
        self.beta = beta

    def levy_flight(self, beta):
        sigma = (np.math.gamma(1 + beta) * np.sin(np.pi * beta / 2) /
                 (np.math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u = np.random.normal(0, sigma, size=self.decision_variables)
        v = np.random.normal(0, 1, size=self.decision_variables)
        step = u / abs(v) ** (1 / beta)
        return step

    def global_pollination(self, flower):
        step_size = self.gamma * self.levy_flight(self.beta)
        new_position = flower.position + step_size * \
            (flower.position - self.global_best.position)
        return new_position

    def local_pollination(self, flower):
        flower_j = np.random.choice(self.population)
        epsilon = np.random.uniform(
            low=0, high=1, size=self.decision_variables)
        new_position = flower.position + epsilon * \
            (flower_j.position - flower.position)
        return new_position

    def pollination(self, flower):
        r = np.random.uniform(low=0, high=1)
        if r < self.switch_prob:
            new_position = self.global_pollination(flower)
        else:
            new_position = self.local_pollination(flower)
        return np.clip(new_position, self.lower_bound, self.upper_bound)

    def find_global_best(self):
        for flower in self.population:
            if flower.fitness < self.global_best.fitness:
                self.global_best = flower

    def optimize(self):
        self.global_best = self.population[0]
        self.global_best.fitness = self.fitness_function(self.global_best.position)
        sa = SimulatedAnnealing(self.fitness_function, self.global_best.position, initial_temperature=10, final_temperature=0.001, cooling_rate=0.9)

        for g in range(self.generations):
            for flower in self.population:
                flower.fitness = self.fitness_function(flower.position)
                self.find_global_best()

                new_position = self.pollination(flower)
                new_fitness = self.fitness_function(new_position)

                if sa.accept(new_fitness):
                    flower.position = new_position
                    flower.fitness = new_fitness

            sa.cool_down()  # cool down the temperature after each generation
            self.find_global_best()
            #print(f"Generation: {g}, Best fitness: {self.global_best.fitness}")

        return self.global_best

# Fitness Function
class FitnessFunction:
    def __init__(self, tasks, servers):
        self.tasks = tasks
        self.servers = servers
        self.makespans = {}

    def __call__(self, solution):
        task_times = []
        for i, task in enumerate(self.tasks):
            server = self.servers[solution[i]]
            processing_time = task['duration'] / server.dc_config['processing_power']
            task_times.append(server.latest_task_end_time + processing_time)
            server.latest_task_end_time += processing_time

        max_time = max(task_times)
        self.makespans.setdefault(len(self.tasks), []).append(max_time)
        return max_time

    def get_average_cost(self):
        return np.mean([server.dc_config['cost'] for server in self.servers])

    def get_average_utilization(self):
        return np.mean([server.latest_task_end_time for server in self.servers])

# Initialize Environment and Servers
env = simpy.Environment()
edge_servers = [Server(env, dc) for dc in edge_datacenters]
cloud_servers = [Server(env, dc) for dc in cloud_datacenters]


# Optimization
tasks = generate_tasks()  # Generate tasks using the new function
# Include tasks in your optimization logic

print(tasks)

[{'duration': 615, 'required_ram': 0.3355895025621014, 'required_storage': 0.2712331399239202, 'required_bandwidth': 0.6229464777170808, 'file_size': 0.2838036282602834, 'output_size': 0.8817255591587533}, {'duration': 223, 'required_ram': 0.25086013361414594, 'required_storage': 0.12714488658933992, 'required_bandwidth': 0.1464661588547776, 'file_size': 0.9240016619070262, 'output_size': 0.39104282778211463}, {'duration': 460, 'required_ram': 0.6018287316177796, 'required_storage': 0.6545688596963082, 'required_bandwidth': 0.1717674463537456, 'file_size': 0.7925715591539453, 'output_size': 0.22648756219362368}, {'duration': 655, 'required_ram': 0.9954273358842007, 'required_storage': 0.9761647130618428, 'required_bandwidth': 0.8558523508551075, 'file_size': 0.4671980385336114, 'output_size': 0.8336360662321514}, {'duration': 327, 'required_ram': 0.9322565333436318, 'required_storage': 0.9633639901658231, 'required_bandwidth': 0.5457860871489045, 'file_size': 0.4718744679523119, 'outpu